In [1]:
from traj_py import XPStraj

In [2]:
import bluesky.preprocessors as bpp
import bluesky.plan_stubs as bps

In [3]:
class ScanningExperimentalModule2():
    """ the zero for Ry must be set correctly so that Rx is pointing in the x direction
        once homed, this position i1s at -6.0
    """
    x = EpicsMotor('XF:16IDC-ES:Scan2{Ax:sX}Mtr', name='ss2_x')
    x1 = EpicsMotor('XF:16IDC-ES:Scan2{Ax:X}Mtr', name='ss2_x1')
    y = EpicsMotor('XF:16IDC-ES:Scan2{Ax:sY}Mtr', name='ss2_y')
    z = EpicsMotor('XF:16IDC-ES:InAir{Mscp:1-Ax:F}Mtr', name='focus')
    # this is the Standa stepper stage
    rx = EpicsMotor('XF:16IDC-ES:Scan2{Ax:RX1}Mtr', name='ss2_rx')
    ry = EpicsMotor('XF:16IDC-ES:Scan2{Ax:RY}Mtr', name='ss2_ry')    
    
ss2 = ScanningExperimentalModule2()
    
xps_trj = XPStraj('10.16.2.100', 'scan', 'test')

In [4]:
xps_trj_motors = {'scan.rY': ss2.ry, 
                  'scan.Y': ss2.y, 
                  'scan.X': ss2.x}

DETS = [pil1M_ext, pilW1_ext, pilW2_ext, em1, em2]

In [6]:
def raster(detectors, exp_time, fast_axis, f_start, f_end, Nfast, 
           slow_axis=None, s_start=0, s_end=0, Nslow=1, monitors=[em1, em2], md=None):
    """ raster scan in fly mode using detectors with exposure time of exp_time
        detectors must be a member of pilatus_detectors_ext
        fly on the fast_axis, step on the slow_axis, both specified as Ophyd motors
        the fast_axis must be one of member of xps_trj.motors, for now this is hard-coded
        the specified positions are relative to the current position
        for the fast_axis are the average positions during detector exposure 
        
        use it within the run engine: RE(raster(...))
    """
    if not set(detectors).issubset(pilatus_detectors_ext):
        raise Exception("only pilatus_detectors_ext can be used in this raster scan.")
    if fast_axis not in xps_trj_motors.values():
        raise Exception("the fast_axis is not supported in this raster scan: ", fast_axis.name)
    fast_axis_name = list(xps_trj_motors.keys())[list(xps_trj_motors.values()).index(fast_axis)]
    # 
    step_size = (f_end-f_start)/(Nfast-1)
    dt = exp_time + 0.005    # exposure_period is 5ms longer than exposure_time, as defined in Pilatus
    xps_trj.define_traj(fast_axis_name, Nfast-1, step_size, dt)
    p0_fast = fast_axis.position
    ready_pos = {}
    ready_pos[True] = p0_fast+f_start-xps_trj.traj_par['rampup_distance']-step_size/2
    ready_pos[False] = p0_fast+f_end+xps_trj.traj_par['rampup_distance']+step_size/2
    
    if slow_axis is not None:
        p0_slow = slow_axis.position
        pos_s = p0_slow+np.linspace(s_start, s_end, Nslow)
    else:
        Nslow = 1
    
    pilatus_ct_time(exp_time)
    set_pil_num_images(Nfast*Nslow)
    print('setting up to collect %d exposures of %.2f sec ...' % (Nfast*Nslow, exp_time))
    
    motor_names = [fast_axis.name, slow_axis.name]
    motors = [fast_axis, slow_axis]
    scan_shape = [Nfast, Nslow]
    _md = {'shape': tuple(scan_shape),
           'plan_args': {'detectors': list(map(repr, detectors))},
           'plan_name': 'raster',
           'plan_pattern': 'outer_product',
           'motors': tuple(motor_names),
           'hints': {},
           }
    _md.update(md or {})
    _md['hints'].setdefault('dimensions', [(('time',), 'primary')])
        
    @bpp.stage_decorator([xps_trj] + detectors)
    @bpp.run_decorator(md=_md)
    @fast_shutter_decorator()
    def inner(fast_axis, ready_pos, slow_axis, Nslow, pos_s):
        running_forward = True
        for i in range(Nslow):
            if slow_axis is not None:
                yield from mov(fast_axis, ready_pos[running_forward], slow_axis, pos_s[i])
            else:
                yield from mov(fast_axis, ready_pos[running_forward])
            yield from wait()
            xps_trj.select_forward_traj(running_forward)
            yield from bps.kickoff(xps_trj, wait=True)
            yield from bps.complete(xps_trj, wait=True)
            yield from bps.collect(xps_trj)
            running_forward = not running_forward

    #yield from bps.open_run()
    #for mo in monitors:
    #    yield from bps.monitor(mo)
    yield from inner(fast_axis, ready_pos, slow_axis, Nslow, pos_s)
    #for mo in monitors:
    #    yield from bps.unmonitor(mo)
    #yield from bps.close_run()
    
    if slow_axis is not None:
        yield from mov(fast_axis, p0_fast, slow_axis, p0_slow)
    else:
        yield from mov(fast_axis, p0_fast)


In [7]:
for m in raster([pil1M_ext], 0.2, ss2.x, -0.1, 0.1, 21, ss2.y, -0.1, 0.1, 11):
    print(m)

setting up to collect 231 exposures of 0.20 sec ...
stage: (XPStraj(prefix='', name='test', read_attrs=[], configuration_attrs=[])), (), {}
stage: (LIXPilatusExt(prefix='XF:16IDC-DT{Det:SAXS}', name='pil1M_ext', read_attrs=['file', 'file.file_path', 'file.file_number', 'file.file_name', 'file.file_template'], configuration_attrs=['cam', 'cam.acquire_period', 'cam.acquire_time', 'cam.image_mode', 'cam.manufacturer', 'cam.model', 'cam.num_exposures', 'cam.trigger_mode', 'file'])), (), {}
open_run: (None), (), {'shape': (21, 11), 'plan_args': {'detectors': ["LIXPilatusExt(prefix='XF:16IDC-DT{Det:SAXS}', name='pil1M_ext', read_attrs=['file', 'file.file_path', 'file.file_number', 'file.file_name', 'file.file_template'], configuration_attrs=['cam', 'cam.acquire_period', 'cam.acquire_time', 'cam.image_mode', 'cam.manufacturer', 'cam.model', 'cam.num_exposures', 'cam.trigger_mode', 'file'])"]}, 'plan_name': 'raster', 'plan_pattern': 'outer_product', 'motors': ('ss2_x', 'ss2_y'), 'hints': {'dim

In [8]:
login('test', 'test', 'test')

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /GPFS/xf16id/exp_path/test/test/log-test.2018Nov14_14:18:43
Mode           : backup
Output logging : True
Raw input log  : True
Timestamping   : True
State          : active


In [9]:
RE(raster([pil1M_ext, pilW1_ext, pilW2_ext], 0.2, ss2.x, -0.1, 0.1, 21, ss2.y, -0.1, 0.1, 11))

setting up to collect 231 exposures of 0.20 sec ...
pil1M_ext staging
pil1M_ext stage sigs updated
resetting file number for  pil1M_ext
pil1M_ext super staged
pil1M_ext checking armed status
pil1M_ext staged
pilW1_ext staging
pilW1_ext stage sigs updated
resetting file number for  pilW1_ext
pilW1_ext super staged
pilW1_ext checking armed status
pilW1_ext staged
pilW2_ext staging
pilW2_ext stage sigs updated
resetting file number for  pilW2_ext
pilW2_ext super staged
pilW2_ext checking armed status
pilW2_ext staged
Transient Scan ID: 1     Time: 2018/11/14 14:18:50
Persistent Unique Scan ID: 'b1aed52b-3e6a-4c3d-a61f-0a4c01a21cd8'
Scan ID: 1
Unique ID: b1aed52b-3e6a-4c3d-a61f-0a4c01a21cd8
#STARTDOC : 1542223130.368701
#STARTDOC : Wed Nov 14 14:18:50 2018
cannot connect to SR:C16-ID:G1{IVU:1-LEnc}Gap
setting triggering parameters: 3, 23, 0.205
New stream: 'test'
rd: [2.895, 2.9049, 2.915, 2.9249, 2.935, 2.945, 2.955, 2.9651, 2.9749, 2.9851, 2.995, 3.0049, 3.015, 3.0249, 3.035, 3.045, 3.05

('b1aed52b-3e6a-4c3d-a61f-0a4c01a21cd8',)

In [12]:
hdr = db[-1]

In [13]:
hdr.fields()

{'position'}

In [14]:
hdr.table(stream_name='test')

,time,position
seq_num,,
1,2018-11-14 14:19:01.378587961,2.8950
2,2018-11-14 14:19:01.378913641,2.9049
3,2018-11-14 14:19:01.378950596,2.9150
4,2018-11-14 14:19:01.378977299,2.9249
5,2018-11-14 14:19:01.379000425,2.9350
6,2018-11-14 14:19:01.379022360,2.9450
7,2018-11-14 14:19:01.379044771,2.9550
8,2018-11-14 14:19:01.379066944,2.9651
9,2018-11-14 14:19:01.379088402,2.9749


In [15]:
hdr.descriptors

[{'run_start': 'b1aed52b-3e6a-4c3d-a61f-0a4c01a21cd8',
  'data_keys': {'position': {'dtype': 'number',
    'shape': [1],
    'source': 'PVT trajectory readback position'}},
  'time': 1542223141.2224886,
  'uid': 'bab974af-04a8-412a-bd4c-99479e57c483',
  'name': 'test',
  'hints': {'test': {'fields': []}},
  'object_keys': {'test': ['position']}}]